# DNA Fun!

This is a practice project for getting comfortable with string manipulation in Python. It's un-optimised code that doesn't account for a *lot* of things, so it wouldn't work on real genomic sequences. 

Things I want to do:
- Identify a string as a DNA sequence
- Similarly, identify a string as an RNA sequence
- Reverse complement a DNA sequence
- Transcribe DNA to RNA
- Find the positions of all the start and stop codons in a sequence
- Find all open reading frames in a sequence
- Translate a particular ORF into the polypeptide sequence

In [61]:
# Store a DNA sequence and a nonsense sequence
dummy_dna = 'ATGTTCCGTACAAGCCCCTCATTTCAGCTAGAGGGTCAACGTTAAGC'
dummy_else = 'TVTTCCFLGTACAAGCCCMXCTCATTTCPAGCTAGAGGGTSZDC'
dummy_rna = 'UUCCGUACAAGCCCAUGCCUCAUUUCAGCUAGAGGGUCAACGUUAAGC'

DNA identifier:

In [62]:
def is_dna(seq):
    '''
    Given a string, determine if it represents a DNA sequence.
    '''
    assert type(seq) is str, 'please input sequence as string'
    intermediate = seq.lower()
    num_nucs = intermediate.count('a') + intermediate.count('t') + intermediate.count('g') + intermediate.count('c')
    return (num_nucs == len(intermediate))

print(is_dna(dummy_dna))
print(is_dna(dummy_else))

True
False


RNA identifier:

In [50]:
def is_rna(seq):
    '''
    Given a string, determine if it represents a RNA sequence.
    '''
    assert type(seq) is str, 'please input sequence as string'
    intermediate = seq.lower()
    num_nucs = intermediate.count('a') + intermediate.count('u') + intermediate.count('g') + intermediate.count('c')
    return (num_nucs == len(intermediate))

print(is_rna(dummy_rna))
print(is_rna(dummy_dna))

True
False


Reverse complement:

In [65]:
def rev_comp(seq):
    '''
    Returns the reverse complement of a DNA sequence.
    '''
    assert type(seq) is str, 'please input sequence as string'
    assert is_dna(seq) == True, 'the input is not a DNA sequence'
    seq = seq.lower()
    rev_seq = seq[::-1] #reversed
    wc_pair = {'a':'t', 't':'a', 'g':'c', 'c':'g'}
    rev_comp = ''.join([wc_pair.get(i) for i in rev_seq])
    return rev_comp

rev_comp(dummy_dna)

'gcttaacgttgaccctctagctgaaatgaggggcttgtacggaacat'

Transcription:

In [52]:
def transcribe(seq, strand = 'coding'):
    '''
    Returns the RNA transcription of the input DNA sequence.
    User can specify if the DNA sequence entered is on the coding or non-coding strand.
    '''
    assert type(seq) is str, 'please input sequence as string'
    assert is_dna(seq) == True, 'the input is not a DNA sequence'
    
    if strand == 'coding':
        seq = seq.lower()
        return seq.replace('t', 'u')
    
    elif strand == 'ncoding':
        re_co = rev_comp(seq)
        return re_co.replace('t', 'u')


print(transcribe(dummy_dna))  
print(transcribe(dummy_dna, strand = 'ncoding'))

uuccguacaagccccucauuucagcuagagggucaacguuaagc
gcuuaacguugacccucuagcugaaaugaggggcuuguacggaa


Start codons and ORFs:

In [64]:
def find_orfs(seq, seq_type = 'rna', strand = 'coding'):
    '''
    Finds start codons in the input sequence (default type is RNA).
    Outputs the starting index of each start codon.
    Strand argument refers to coding and noncoding strands of DNA (only specify if input is a DNA sequence)
    '''
    import re
    assert type(seq) is str
    
    if seq_type == 'rna':
        assert is_rna(seq) == True, "input doesn't match sequence type"
        rna = seq.lower()  
    elif seq_type == 'dna':
        assert is_dna(seq) == True, "input doesn't match sequence type"
        rna = transcribe(seq, strand = strand)  
    else:
        raise ValueError ('inappropriate sequence type')
        
    M = [m.start() for m in re.finditer('aug', rna)]    
    return M

    
print(find_orfs(dummy_rna, seq_type = 'rna'))
print(find_orfs(dummy_dna, seq_type = 'dna'))

[14]
[0]


Find stop codons:

In [78]:
def find_stops(seq, seq_type = 'rna', strand = 'coding'):
    '''
    Finds stop codons in the input sequence (default type is RNA).
    Outputs the starting index of each stop codon.
    Strand argument refers to coding and noncoding strands of DNA (only specify if input is a DNA sequence)
    '''
    import re
    assert type(seq) is str
    
    if seq_type == 'rna':
        assert is_rna(seq) == True, "input doesn't match sequence type"
        rna = seq.lower()  
    elif seq_type == 'dna':
        assert is_dna(seq) == True, "input doesn't match sequence type"
        rna = transcribe(seq, strand = strand)  
    else:
        raise ValueError ('inappropriate sequence type')
        
    stop_1 = [m.start() for m in re.finditer('uaa', rna)] 
    stop_2 = [m.start() for m in re.finditer('uag', rna)]
    stop_3 = [m.start() for m in re.finditer('uga', rna)]
    
    return stop_1 + stop_2 + stop_3


print(find_stops(dummy_rna, seq_type = 'rna'))
print(find_stops(dummy_dna, seq_type = 'dna'))

[43, 29]
[42, 28]


Reading frames:

In [110]:
def find_frames(seq, seq_type = 'rna', strand = 'coding'):
    starts = find_orfs(seq, seq_type = seq_type, strand = strand)
    ends = find_stops(seq, seq_type = seq_type, strand = strand)
    fragments = [(i,j) for i in starts for j in ends]
    frames = [seq[i[0]:i[1]] for i in fragments if len(seq[i[0]:i[1]])%3 == 0]
    return frames

print(find_frames(dummy_dna, seq_type = 'dna'))
len(find_frames(dummy_dna, seq_type = 'dna')[0])

['ATGTTCCGTACAAGCCCCTCATTTCAGCTAGAGGGTCAACGT']


42

In [116]:
def translate(seq, seq_type = 'rna', strand = 'coding'):
    '''
    Input: RNA (default) or DNA sequence.
    Output: a list of all possible polypeptides encoded by the sequence.
    '''
    assert type(seq) is str
    
    if seq_type == 'rna':
        assert is_rna(seq) == True, "input doesn't match sequence type"
        rna = seq.lower()  
    elif seq_type == 'dna':
        assert is_dna(seq) == True, "input doesn't match sequence type"
        rna = transcribe(seq, strand = strand)  
    else:
        raise ValueError ('inappropriate sequence type')
    
    frames = find_frames(rna)

    genetic_code = {'ugg': 'W',
                    'aug': 'M',
                    'uuu': 'F', 'uuc': 'F',
                    'uau': 'Y', 'uac': 'Y',
                    'cau': 'H', 'cac': 'H',
                    'caa': 'Q', 'cag': 'Q',
                    'aau': 'N', 'aac': 'N',
                    'aaa': 'K', 'aag': 'K',
                    'gau': 'D', 'gac': 'D',
                    'gaa': 'E', 'gag': 'E',
                    'ugu': 'C', 'ugc': 'C',
                    'auu': 'I', 'auc': 'I', 'aua': 'I', 
                    'uaa': 'STOP', 'uag': 'STOP', 'uga': 'STOP',
                    'ggu': 'G', 'ggc': 'G', 'gga': 'G', 'ggg': 'G',
                    'guu': 'V', 'guc': 'V', 'gua': 'V', 'gug': 'V',
                    'ccu': 'P', 'ccc': 'P', 'cca': 'P', 'ccg': 'P', 
                    'acu': 'T', 'acc': 'T', 'aca': 'T', 'acg': 'T',
                    'gcu': 'A', 'gcc': 'A', 'gca': 'A', 'gcg': 'A',
                    'cuu': 'L', 'cuc': 'L', 'cua': 'L', 'cug': 'L', 'uua': 'L', 'uug': 'L',
                    'cgu': 'R', 'cgc': 'R', 'cga': 'R', 'cgg': 'R','aga': 'R', 'agg': 'R',
                    'ucu': 'S', 'ucc': 'S', 'uca': 'S', 'ucg': 'S', 'agu': 'S', 'agc': 'S'}
    from textwrap import wrap

    result = []
    for i in frames:
        codons = wrap(i, 3)
        polypeptide = ''.join([genetic_code.get(c) for c in codons])
        result.append(polypeptide)
        
    return result

print(translate(dummy_rna))
print(translate(dummy_dna, seq_type = 'dna'))

['MPHFS']
['MFRTSPSFQLEGQR']


Done! Now I can start with any short DNA or RNA sequence and find all the possible polypeptide sequences it encodes :)